# Text Acquisition & Ingestion Assignment

In [2]:
!pip install feedparser

     |████████████████████████████████| 81kB 3.6MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp36-none-any.whl size=6066 sha256=f642822111848d2b4885146a97f9b4056194157b7c533cadced9ed76884dd748
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


In [3]:
import json
import requests
import feedparser
from bs4 import BeautifulSoup

### Iterate through the list of article URLs below, scraping the text from each one and saving it to a text file. 

In [52]:
articles = ['http://lite.cnn.io/en/article/h_eac18760a7a7f9a1bf33616f1c4a336d',
            'http://lite.cnn.io/en/article/h_de3f82f17d289680dd2b47c6413ebe7c',
            'http://lite.cnn.io/en/article/h_72f4dc9d6f35458a89af014b62e625ad',
            'http://lite.cnn.io/en/article/h_aa21fe6bf176071cb49e09d422c3adf0',
            'http://lite.cnn.io/en/article/h_8ad34a532921c9076cdc9d7390d2f1bc',
            'http://lite.cnn.io/en/article/h_84422c79110d9989177cfaf1c5f45fe7',
            'http://lite.cnn.io/en/article/h_d010d9580abac3a44c6181ec6fb63d58',
            'http://lite.cnn.io/en/article/h_fb11f4e9d7c5323e75b337d9e9e5e368',
            'http://lite.cnn.io/en/article/h_7b27f0b131067f8ece6238ac559670ab',
            'http://lite.cnn.io/en/article/h_8cae7f735fa9573d470f802063ceffe2',
            'http://lite.cnn.io/en/article/h_72c3668280e82576fcc2602b0fa70c14',
            'http://lite.cnn.io/en/article/h_d20658fb0e20212051cda0e0a7248c8a',
            'http://lite.cnn.io/en/article/h_56611c43d7928120d2ae21666ccc7417',
            'http://lite.cnn.io/en/article/h_bda0394e3c5ee7054ee65c022bca7695']

In [109]:
len(articles)

14

In [95]:
PATH = '/content/WebScrapePractice/'

for i, article in enumerate(articles):

  url = article
  res = requests.get(url)
  con = res.text
  TAGS = ['h2','p']
  soupy = BeautifulSoup(con, 'lxml')
  text_list = [tag.get_text() for tag in soupy.find_all(TAGS)]
  text = ' '.join(text_list)
  
  #print(text)
  with open(PATH + f'article_{i}.txt', 'wb') as f:
    f.write(text.encode())

### Ingest the text files generated via web scraping into a corpus and print the corpus statistics.

In [145]:
def corpus_stats(corpus):
  print('Corpus Statistics')
  print('Number of documents:    ' + str(len(corpus.fileids())))
  print('Number of paragraphs:   ' + str(len(corpus.paras())))
  print('Number of sentences:    ' + str(len(corpus.sents())))
  print('Number of words:        ' + str(len(corpus.words())))
  print('Vocabulary:             ' + str(len(set(w.lower() for w in corpus.words()))))
  print('Avg char per word:      ' + str(round(len(corpus.raw())/len(corpus.words()),1)))
  print('Avg words per sentence: ' + str(round(len(corpus.words())/len(corpus.sents()),1)))

In [128]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [113]:
DOC_PATTERN = r'.*\.txt'
corpus = PlaintextCorpusReader('/content/WebScrapePractice', DOC_PATTERN)

In [146]:
corpus_stats(corpus)

Corpus Statistics
Number of documents:    14
Number of paragraphs:   14
Number of sentences:    590
Number of words:        14159
Vocabulary:             2914
Avg char per word:      5.0
Avg words per sentence: 24.0


### Parse the O'Reilly Radar RSS feed below, extract the text from each post, and save it to a text file.

The content of each post contains HTML tags. Strip those out using the same approach you used for web scraping so that only text is saved to the files.

In [118]:
feed = 'http://feeds.feedburner.com/oreilly/radar/atom'

In [119]:
parsed = feedparser.parse(feed)
posts = parsed.entries

In [120]:
len(posts)

60

In [122]:
posts[0]['summary']

'2020 has been a year of great challenges for so many, but it’s not all negative. Around the world, organizations and their workforces have risen to the occasion, recognizing the importance of expanding their knowledge, taking on new tasks, and bettering themselves both personally and professionally. With the uptick in virtual conferencing, remote work, and, [&#8230;]'

In [124]:
PATH = '/content/rss_info/'

for i, post in enumerate(posts):
  text = post.summary

  with open(PATH + f'post_{i}.txt', 'wb') as f:
    f.write(text.encode())

### Ingest the text files generated via RSS parsing into a corpus and print the corpus statistics.

In [125]:
DOC_PATTERNrss = r'.*\.txt'
corpusrss = PlaintextCorpusReader('/content/rss_info', DOC_PATTERNrss)

In [147]:
corpus_stats(corpusrss)

Corpus Statistics
Number of documents:    60
Number of paragraphs:   60
Number of sentences:    197
Number of words:        4515
Vocabulary:             1467
Avg char per word:      4.9
Avg words per sentence: 22.9


### Make an API call to the Hacker News API to retrieve their Ask, Show, and Job category items. 

- URL: https://hacker-news.firebaseio.com/v0/askstories.json

In [206]:
URLask = 'https://hacker-news.firebaseio.com/v0/askstories.json'
URLshow = 'https://hacker-news.firebaseio.com/v0/showstories.json'
URLjob = 'https://hacker-news.firebaseio.com/v0/jobstories.json'

In [210]:
response_ask = requests.get(URLask).json()
response_show = requests.get(URLshow).json()
response_job = requests.get(URLjob).json()

In [211]:
print(len(response_ask))
print(len(response_show))
print(len(response_job))

92
39
60


In [215]:
ids = []
ids.extend(response_ask)
ids.extend(response_show)
ids.extend(response_job)

In [216]:
len(ids)

191

### Once you have retrieved the item IDs from the URL above, retrieve each item by adding the item ID to the URL below, extract the item's text property, and save the text from each item to disk as its own document.

- URL: https://hacker-news.firebaseio.com/v0/item/ITEM_ID_HERE.json

The content of some items may contain HTML tags. Strip those out using the same approach you used for web scraping so that only text is saved to the files.

In [219]:
URL_ = 'https://hacker-news.firebaseio.com/v0/item/'+str(ids[0])+'.json'
response1 = requests.get(URL_)
results1 = json.loads(response1.content)
results1['text']

'I love asking new colleagues this question, figured I would open it here as well.<p>What are you surprised isn’t being worked on more?'

In [217]:
PATH = '/content/api_test/'
for i in ids:
  URL_Loop = 'https://hacker-news.firebaseio.com/v0/item/'+ str(i) + '.json'
  responselp = requests.get(URL_Loop).json()
  if 'text' in responselp:
    soup = BeautifulSoup(responselp['text'])
    text = soup.text
    with open(PATH + f'api_article_{i}.txt', 'wb') as f:
      f.write(text.encode())
  

### Ingest the text files generated via API into a corpus and print the corpus statistics.

In [218]:
DOC_PATTERNapi = r'.*\.txt'
corpusapi = PlaintextCorpusReader('/content/api_test', DOC_PATTERNapi)
corpus_stats(corpusapi)

Corpus Statistics
Number of documents:    82
Number of paragraphs:   82
Number of sentences:    303
Number of words:        8834
Vocabulary:             2217
Avg char per word:      4.7
Avg words per sentence: 29.2


# Notes

In [ ]:
# Notes

In [5]:
url = 'https://quotes.toscrape.com/'
response = requests.get(url)
content = response.text

In [ ]:
content

In [7]:
soup = BeautifulSoup(content)

In [ ]:
print(soup.prettify())

In [16]:
## Print with tags
#soup.find('span', class_='text')
#soup.find('span', {'class': 'text'})



## Print no tags just text
#soup.find('span', {'itemprop': 'text'}).text
soup.find('span', {'itemprop': 'text'}).get_text()

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [18]:
quotes = soup.find_all('span', {'class': 'text'})

In [ ]:
for quote in quotes:
  print(quote.get_text())
  print('------')

In [23]:
my_quotes = [quote.text for quote in quotes]

In [ ]:
my_quotes

In [ ]:
url1 = 'https://books.toscrape.com/'
response1 = requests.get(url1)
content1 = response1.text
soup1 = BeautifulSoup(content1, 'html.parser') # 'lxml'
print(soup1.prettify())

In [30]:
article = soup1.find('article', {'class' : 'product_pod'})

In [ ]:
article

In [32]:
article.find('p', {'class': 'price_color'}).text

'Â£51.77'

In [33]:
article.find('h3').text #title

'A Light in the ...'

In [35]:
feed = 'http://feeds.feedburner.com/oreilly/radar/atom'
parsed = feedparser.parse(feed)
posts = parsed.entries

In [38]:

len(parsed.entries)

60

In [ ]:
# Looking at first post
#posts[0].summary
posts[0]['summary']

In [ ]:
for post in posts:
  print(post.summary)
  print('----')

## Saving into a corpus

In [45]:
PATH = '/content/rss_posts/'

for i, post in enumerate(posts):
  text = post.summary

  with open(PATH + f'article_{i}.txt', 'wb') as f:
    f.write(text.encode())

## Reading previously saved corpus

In [48]:
DOC_PATTERN = r'.*\.txt'
news_corpus = PlaintextCorpusReader('/content/rss_posts', DOC_PATTERN)

In [ ]:
news_corpus.fileids()

In [51]:
news_corpus.raw(news_corpus.fileids()[0])

'2020 has been a year of great challenges for so many, but it’s not all negative. Around the world, organizations and their workforces have risen to the occasion, recognizing the importance of expanding their knowledge, taking on new tasks, and bettering themselves both personally and professionally. With the uptick in virtual conferencing, remote work, and, [&#8230;]'